In [1]:
#pip install tensorflow opencv-python matplotlib

Note: you may need to restart the kernel to use updated packages.


In [176]:
#!pip list

In [1]:
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np

# Removing unclear images

In [2]:
data_dir = 'data' 

In [3]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [4]:
os.listdir(data_dir)

['000',
 '001',
 '002',
 '003',
 '004',
 '005',
 '006',
 '007',
 '008',
 '009',
 '010',
 '011',
 '012',
 '013',
 '014',
 '015',
 '016',
 '017',
 '018',
 '019',
 '020',
 '021',
 '022',
 '023',
 '024',
 '025',
 '026',
 '027',
 '028',
 '029',
 '030',
 '031',
 '032',
 '033',
 '034',
 '035',
 '036',
 '037',
 '038',
 '039',
 '040',
 '041',
 '042',
 '043',
 '044',
 '045',
 '046',
 '047',
 '048',
 '049',
 '050',
 '051',
 '052',
 '053',
 '054',
 '055',
 '056',
 '057',
 '058',
 '059',
 '060',
 '061',
 '062',
 '063',
 '064',
 '065',
 '066',
 '067',
 '068',
 '069',
 '070',
 '071',
 '072',
 '073',
 '074',
 '075',
 '076',
 '077',
 '078',
 '079',
 '080',
 '081',
 '082',
 '083',
 '084',
 '085',
 '086',
 '087',
 '088',
 '089',
 '090',
 '091',
 '092',
 '093',
 '094',
 '095',
 '096',
 '097',
 '098',
 '099',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123']

In [10]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                #os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# Load Data

In [5]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

Found 62496 files belonging to 124 classes.


In [6]:
data

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

# Scaling Data

In [8]:
data = data.map(lambda x,y: (x/255, y))

# Split Data

In [9]:
len(data)

1953

In [10]:
#split into train and test

train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)+1

In [11]:
print(train_size)
print(val_size)
print(test_size)

1367
390
196


In [12]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# Deep Learning Model using CNN

In [13]:
train

<TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [22]:
model = Sequential()

In [23]:
#Add convolution and max-pooling layer
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(124, activation='softmax'))

In [24]:
model.compile(optimizer = 'rmsprop',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 16)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 16)      

# Training

In [26]:
logdir='logs'

In [27]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [28]:
hist = model.fit(train, epochs=15, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/15
1367/1367 [==============================] - 1198s 875ms/step - loss: 4.8211 - accuracy: 0.0074 - val_loss: 4.8215 - val_accuracy: 0.0074
Epoch 2/15
1367/1367 [==============================] - 1156s 845ms/step - loss: 4.8209 - accuracy: 0.0085 - val_loss: 4.8222 - val_accuracy: 0.0062
Epoch 3/15
1367/1367 [==============================] - 1144s 836ms/step - loss: 4.8209 - accuracy: 0.0083 - val_loss: 4.8225 - val_accuracy: 0.0063
Epoch 4/15
1367/1367 [==============================] - 1089s 796ms/step - loss: 4.8209 - accuracy: 0.0077 - val_loss: 4.8227 - val_accuracy: 0.0066
Epoch 5/15
1367/1367 [==============================] - 1153s 842ms/step - loss: 4.8209 - accuracy: 0.0082 - val_loss: 4.8227 - val_accuracy: 0.0065
Epoch 6/15
1367/1367 [==============================] - 1145s 837ms/step - loss: 4.8209 - accuracy: 0.0081 - val_loss: 4.8228 - val_accuracy: 0.0064
Epoch 7/15
1367/1367 [==============================] - 1108s 810ms/step - loss: 4.8209 - accuracy: 0.0081

KeyboardInterrupt: 

# Evaluation

In [26]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [27]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [28]:
import numpy as np
from sklearn.metrics import r2_score

for batch in test.as_numpy_iterator():
    cmp = [] 
    X, y = batch
    yhat = model.predict(X)

    for j in range(len(yhat)):
        cmp.append(np.argmax(yhat[j]))
    
    print(y)
    print(cmp)

    print(r2_score(y,cmp))

    pre.update_state(y, cmp)
    re.update_state(y, cmp)
    acc.update_state(y, cmp)

1/1 [==============================] - 0s 291ms/step
[ 74   4  81   4   6 118 103  67 107  83  18  18  84  77   2  68  85  47
  95  31  78 103 109  44 103   5   4  17  85  11   5  11]
[74, 4, 81, 4, 6, 118, 103, 67, 107, 83, 19, 18, 84, 77, 2, 68, 85, 47, 95, 31, 78, 103, 109, 44, 103, 5, 4, 17, 85, 10, 5, 10]
0.9999408420760734
1/1 [==============================] - 0s 180ms/step
[ 53  51  55  97 102  59  64  67  86  51 110 102  71   5  82  70 117 104
  53  85  13  94   2  20  84   6  34  97  12  80  88  24]
[53, 51, 55, 97, 102, 59, 64, 49, 86, 51, 110, 102, 71, 5, 82, 70, 117, 113, 65, 85, 13, 94, 2, 20, 84, 39, 34, 97, 12, 80, 88, 24]
0.9547249603532448
1/1 [==============================] - 0s 169ms/step
[ 21   6 112  46  48   7 110  34  60  13  35  57  60  51 111  28   9  20
 117 118   8  75  32  34 112  11 123 102  18  71  28  60]
[21, 6, 112, 46, 50, 7, 110, 34, 60, 13, 35, 57, 60, 51, 116, 28, 9, 20, 117, 118, 8, 75, 32, 34, 112, 10, 123, 102, 18, 71, 28, 60]
0.999373936588906

In [29]:
print(f"precision : {pre.result().numpy()}")
print(f"Recall : {re.result().numpy()}")
print(f"accuracy : {acc.result().numpy()}")

precision : 1.0
Recall : 0.9993573427200317
accuracy : 0.9030612111091614


# Save the Model

In [29]:
model.save(os.path.join('models','imageclassifier.h5'))